This notebook aims to show how the learning rate scheduler looks like throughout training.

Make sure to adapt the chained scheduler below to the scheduler that is actually used.

In [10]:
import sys
sys.path.append("../src")

In [11]:
from ds.models import THGStrainStressCNN
from types import SimpleNamespace
from torch.optim.lr_scheduler import (
    ChainedScheduler,
    CosineAnnealingWarmRestarts,
    LinearLR,
)
import numpy as np
from torch.optim import Adam
import plotly.graph_objects as go

In [12]:
class NestedNamespace(SimpleNamespace):
    """Sot that the stub config dictionary can be accessed by the model
    as if it was the hydra config file.
    """
    def __init__(self, dictionary, **kwargs):
        super().__init__(**kwargs)
        for key, value in dictionary.items():
            if isinstance(value, dict):
                self.__setattr__(key, NestedNamespace(value))
            else:
                self.__setattr__(key, value)

In [13]:
cfg = {
    "params": {
        "model": {
            "n_nodes": 64,
            "num_output_features": 3
        }
}}
cfg = NestedNamespace(cfg)

In [14]:
model = THGStrainStressCNN(cfg)

In [15]:
optimizer = Adam(params=model.parameters(), lr=0.0007969454818643932,weight_decay=2.368863950364079e-05)

In [16]:
scheduler = ChainedScheduler([
    LinearLR(
                optimizer=optimizer, start_factor=0.1, end_factor=1, total_iters=10
            ),
    CosineAnnealingWarmRestarts(optimizer, T_0=92, T_mult=2),
])

epochs = np.arange(3000)
lr_array = np.zeros_like(epochs, dtype=float)
for epoch in epochs:
    lr = scheduler.get_last_lr()[0]
    lr_array[epoch] = lr

    optimizer.step()
    scheduler.step()

In [17]:
fig = go.Figure(data=go.Scatter(x=epochs, y=lr_array))
fig.show()